<a href="https://colab.research.google.com/github/joshual777/PatronesProyecto3/blob/feature%2FHipotesis-1/Leo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:

import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import precision_score, f1_score, roc_auc_score

# Cargar el conjunto de datos Plant Village
data, info = tfds.load('plant_village', split='train[:5%]', shuffle_files=True, with_info=True)



In [12]:
# Dividir el conjunto de datos en datos sin etiquetas y datos con etiquetas
data_unlabeled = data.map(lambda x: x['image']).take(40000)  # 80% de los datos sin etiquetas
data_labeled = data.skip(40000)  # 20% de los datos con etiquetas

# Preprocesamiento de datos sin etiquetas
def preprocess_unlabeled(image):
    image = tf.image.resize(image, (64, 64)) / 255.0
    return image, image

data_unlabeled = data_unlabeled.map(preprocess_unlabeled)

# Construir y entrenar el autoencoder
input_img = Input(shape=(64, 64, 3))
x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer=Adam(), loss='mse')
autoencoder.fit(data_unlabeled.batch(64), epochs=10)

Epoch 1/10
43/43 [==============================] - 26s 582ms/step - loss: 0.0297
Epoch 2/10
43/43 [==============================] - 23s 539ms/step - loss: 0.0129
Epoch 3/10
43/43 [==============================] - 23s 529ms/step - loss: 0.0089
Epoch 4/10
43/43 [==============================] - 25s 582ms/step - loss: 0.0076
Epoch 5/10
43/43 [==============================] - 24s 546ms/step - loss: 0.0071
Epoch 6/10
43/43 [==============================] - 23s 530ms/step - loss: 0.0068
Epoch 7/10
43/43 [==============================] - 23s 532ms/step - loss: 0.0065
Epoch 8/10
43/43 [==============================] - 23s 536ms/step - loss: 0.0062
Epoch 9/10
43/43 [==============================] - 25s 585ms/step - loss: 0.0060
Epoch 10/10
43/43 [==============================] - 23s 533ms/step - loss: 0.0059


In [16]:
# Preprocesamiento de datos con etiquetas
def preprocess_labeled(element):
    image = tf.image.resize(element['image'], (64, 64)) / 255.0
    label = tf.one_hot(element['label'], depth=38)
    return image, label

data_labeled = data_labeled.map(preprocess_labeled)

# Dividir los datos con etiquetas en entrenamiento y prueba
train_size = int(0.8 * info.splits['train'].num_examples)
train_data = data_labeled.take(train_size)
test_data = data_labeled.skip(train_size)

# Construir y entrenar el clasificador desde cero
classifier_from_scratch = tf.keras.Sequential([
    Flatten(),
    Dense(256, activation='relu'),
    Dense(38, activation='softmax')
])


classifier_from_scratch.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'], run_eagerly=True)
classifier_from_scratch.fit(train_data.batch(64), epochs=10, validation_data=test_data.batch(64))

# Construir y entrenar el clasificador basado en el autoencoder
encoder = Model(autoencoder.input, autoencoder.get_layer('max_pooling2d_1').output)

classifier_with_autoencoder = tf.keras.Sequential([
    encoder,
    Flatten(),
    Dense(256, activation='relu'),
    Dense(38, activation='softmax')
])

classifier_with_autoencoder.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
classifier_with_autoencoder.fit(train_data.batch(64), epochs=10, validation_data=test_data.batch(64))


TypeError: ignored

In [ ]:
# Calcular las métricas en el conjunto de prueba
test_images, test_labels = zip(*test_data.batch(64))
test_images = tf.concat(test_images, axis=0)
test_labels = tf.concat(test_labels, axis=0)

# Métricas para el clasificador desde cero
predictions_from_scratch = classifier_from_scratch.predict(test_images)
predicted_labels_from_scratch = tf.argmax(predictions_from_scratch, axis=1)
true_labels = tf.argmax(test_labels, axis=1)

accuracy_from_scratch = tf.reduce_mean(tf.cast(tf.equal(predicted_labels_from_scratch, true_labels), dtype=tf.float32))
precision_from_scratch = precision_score(true_labels, predicted_labels_from_scratch, average='macro')
f1_score_from_scratch = f1_score(true_labels, predicted_labels_from_scratch, average='macro')
auc_from_scratch = roc_auc_score(test_labels, predictions_from_scratch, average='macro')

# Métricas para el clasificador basado en el autoencoder
predictions_with_autoencoder = classifier_with_autoencoder.predict(test_images)
predicted_labels_with_autoencoder = tf.argmax(predictions_with_autoencoder, axis=1)

accuracy_with_autoencoder = tf.reduce_mean(tf.cast(tf.equal(predicted_labels_with_autoencoder, true_labels), dtype=tf.float32))
precision_with_autoencoder = precision_score(true_labels, predicted_labels_with_autoencoder, average='macro')
f1_score_with_autoencoder = f1_score(true_labels, predicted_labels_with_autoencoder, average='macro')
auc_with_autoencoder = roc_auc_score(test_labels, predictions_with_autoencoder, average='macro')

print("Metrics for classifier trained from scratch:")
print("Accuracy:", accuracy_from_scratch.numpy())
print("Precision:", precision_from_scratch)
print("F1 Score:", f1_score_from_scratch)
print("AUC:", auc_from_scratch)

print("Metrics for classifier with autoencoder:")
print("Accuracy:", accuracy_with_autoencoder.numpy())
print("Precision:", precision_with_autoencoder)
print("F1 Score:", f1_score_with_autoencoder)
print("AUC:", auc_with_autoencoder)